In [1]:
!pip install tensorflow opencv-python numpy matplotlib


In [5]:
import zipfile
import os

# Path to your uploaded zip file
zip_file_path = '/content/archive (9).zip'

# Define the destination folder where files will be extracted
extract_path = '/content/extracted_data'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Define the path to the extracted dataset
dataset_dir = '/content/extracted_data'  # Update with your extracted dataset path
train_dir = '/path/to/extracted/train'
val_dir = '/path/to/extracted/val'

# Step 1: Split dataset into train and validation sets
def split_dataset(dataset_dir, train_dir, val_dir, test_size=0.2):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    gesture_classes = os.listdir(dataset_dir)

    for gesture_class in gesture_classes:
        class_path = os.path.join(dataset_dir, gesture_class)

        # Skip if it's not a directory
        if not os.path.isdir(class_path):
            continue

        # Get all image files in the class folder
        images = os.listdir(class_path)

        # Only split if there are enough images
        if len(images) < 2:  # Ensure there's at least 1 for training and 1 for validation
            print(f"Skipping class '{gesture_class}' due to insufficient samples.")
            continue

        # Split into train and validation
        train_images, val_images = train_test_split(images, test_size=test_size, random_state=42)

        # Create subdirectories in train and val folders
        train_class_dir = os.path.join(train_dir, gesture_class)
        val_class_dir = os.path.join(val_dir, gesture_class)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        # Move images to corresponding train and validation folders
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_class_dir, img))
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(val_class_dir, img))

# Split the dataset
split_dataset(dataset_dir, train_dir, val_dir)

# Image parameters
img_width, img_height = 64, 64  # Resize image
batch_size = 32  # Batch size for training
epochs = 20  # Number of epochs

# Step 2: Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Check if the data generators have loaded correctly
print(f"Training samples: {train_generator.samples}, Validation samples: {validation_generator.samples}")

# Step 3: Build the CNN Model
model = Sequential()

# First Convolution + MaxPooling layer
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolution + MaxPooling layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolution + MaxPooling layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add Dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))  # Number of output classes

# Step 4: Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Callbacks for saving the best model and stopping early
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Step 5: Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping]
)

# Step 6: Evaluate and Visualize the Results
# Plot training & validation accuracy and loss
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Save the final model
model.save('gesture_recognition_model.keras')

# Model Evaluation
score = model.evaluate(validation_generator, verbose=1)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

print(f"Training samples: {train_generator.samples}, Validation samples: {validation_generator.samples}")




Skipping class 'HandGesture' due to insufficient samples.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Training samples: 0, Validation samples: 0


ValueError: Must provide at least one structure